# Instructions for Use
## 1) Download XLSX files for the indicator(s) and upload them to the appropriate indicator folder(s) in the [parent folder](https://drive.google.com/drive/folders/1ikgu5-ArNq0tqLZM_0oXBpi-x_kBlVn8).
## 2) Open each file in Drive, click File > Save as Google Sheets. Once created, delete the original XLSX file.
## 3) Edit cell 4 with the indicator(s) and academic year(s) you wish to upload.

In [3]:
import pandas as pd
from spswarehouse.warehouse import Warehouse
from spswarehouse.googledrive import GoogleDrive
from spswarehouse.googlesheets import GoogleSheets as gs
from spswarehouse.table_utils import *

Warehouse.execute("USE ROLE dataops")

(<sqlalchemy.engine.base.Connection at 0x23c3b8f2610>,
 <sqlalchemy.engine.cursor.LegacyCursorResult at 0x23c435edeb0>)

In [13]:
def load_imports(indicator_list, academic_year, folder_map):
    res = {}
    for indicator in indicator_list:
        d = {}
        folder = folder_map[indicator]
        file_list = GoogleDrive.ListFile({'q': f"'{folder}' in parents"}).GetList()

        #filter for academic year if applicable
        if academic_year != 'all':
            file_list = [x for x in file_list if academic_year in x['title']]

        #load in each file to a dataframe with file name as key
        for f in file_list:
            d[f['title']] = f['id']
        
        res[indicator] = d
    return res

def generate_subfolder_dict(folder):
    folder_list = GoogleDrive.ListFile({'q': f"'{folder}' in parents"}).GetList()
    d = {}
    for f in folder_list:
        d[f['title']] = f['id']
    return d

def pull_site_cds_codes():
    query = """
    select distinct site_name,
    state_site_id
    from public.sites
    where state = 'CA'
    and state_site_id is not null
    """
    
    return Warehouse.read_sql(query)

def generate_cds_list(state_cds):
    return pd.concat([pull_site_cds_codes(), pd.DataFrame(state_cds)])

def load_df_from_gs(fid):
    return pd.DataFrame(gs.open_by_key(fid).sheet1.get_all_records(numericise_ignore=[1]))

def load_and_filter_df(fileid, cds_df):
    #load up the full table
    df = load_df_from_gs(fileid)
    print('Loaded in', len(df), 'records')
    dx = pd.merge(cds_df, df, how='left', left_on='state_site_id', right_on='cds', copy=False)
    print('Filtered down to', len(dx), 'records')
    return dx.iloc[:,2:]

In [14]:
#these stay the same
parent_folder = '1ikgu5-ArNq0tqLZM_0oXBpi-x_kBlVn8'
state_cds = {'site_name':['State of California',
                         'West Contra Costa USD',
                         'Jefferson Union HSD',
                         'Sequoia Union HSD',
                         'Santa Clara COE'],
            'state_site_id': ['00000000000000', 
                             '07617960000000',
                             '41689240000000',
                             '41690620000000',
                             '43104390000000']}

#modify these as needed
indicator_list = ['ela']
academic_year = 'all' #specify year in YYYY format if want just one year, defualts to all

In [19]:
#generate some intermediary things
folder_map = generate_subfolder_dict(parent_folder)
m = load_imports(indicator_list, academic_year, folder_map)
cds_df = generate_cds_list(state_cds)

In [20]:
cds_df

,site_name,state_site_id
0,Summit Public School: Tahoma,43104390123794
1,Summit Preparatory Charter High School,41690620112722
2,Everest Public High School,41690620119503
3,Summit Public School: Shasta,41689240127548
4,Summit Public School: K2,07100740129684
5,Summit Public School: Tamalpais,07617960133637
0,State of California,00000000000000
1,West Contra Costa USD,07617960000000
2,Jefferson Union HSD,41689240000000
3,Sequoia Union HSD,41690620000000


In [ ]:
#load each raw file from sheets and filter just for our sites + state
filt = {}

for indicator, dx in m.items():
    print('Loading and Filtering Records for', indicator)
    res = {}
    for export, fileid in dx.items():
        print('Pulling records from export', export)
        res[export] = load_and_filter_df(fileid, cds_df)
    
    filt[indicator] = res
    print('=========================================================')

Loading and Filtering Records for ela
Pulling records from export eladownload2022
Loaded in 169237 records
Filtered down to 181 records
Pulling records from export eladownload2018
Loaded in 148838 records
Filtered down to 169 records
Pulling records from export eladownload2019
Loaded in 165532 records
Filtered down to 177 records
Pulling records from export elapratedownload2019
Loaded in 155216 records
Filtered down to 163 records
Pulling records from export elapratedownload2018


In [78]:
#upload to warehouse
schema_name = 'state_dashboard_ca'

for indicator, dx in filt.items():
    print('Uploading records for', indicator)
    for export, df in dx.items():
        print('Uploading export', export)
        table_name = export
        try:
            Warehouse.execute(f"SELECT 1 FROM {schema_name}.{table_name}")
            print(f"{schema_name}.{table_name} already exists")
        except:
            create_sql = create_table_stmt(table_name, schema_name, dataframe=df, force_string=True)
            Warehouse.execute(create_sql)
            print(f"{schema_name}.{table_name} created")
        
        Warehouse.execute(f"DELETE FROM {schema_name}.{table_name}") #delete first in case have to redo
        table_reflect = Warehouse.reflect(table_name, schema_name)
        upload_to_warehouse(table_reflect, dataframe=df, force_string=True)

Uploading records for ela
Uploading export eladownload2022
state_dashboard_ca.eladownload2022 created
107 rows to insert
Inserted 107 rows to state_dashboard_ca.eladownload2022
